In [1]:
import re,string
import json
import jsonpickle
import os

input_fname = "original_tweets.json"
output_fname = "processed_data.json"
data_dir_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'all_data'))
input_fname_path = os.path.abspath(os.path.join(data_dir_path, input_fname))
output_fname_path = os.path.abspath(os.path.join(data_dir_path, output_fname))

#read the contents of the file into a list
with open(input_fname_path) as f:
    content = f.readlines()
    
#set up the regex for preprocessing
punctuation_regex = re.compile('[%s]' % re.escape(string.punctuation))
digits_regex = re.compile('[%s]' % re.escape(string.digits))
emoji_regex = re.compile(r"["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002700-\U000027FF"
        u"\U00002580-\U0000267F"
        u"\U00002300-\U0000237F"
        u"\U00002400-\U0000247F" 
        u"\U00002B00-\U00002B7F"
                           "]+", flags=re.UNICODE)
url_regex = re.compile('(\w+:\/\/\S+)')     
handle_regex = re.compile('(@[A-Za-z0-9_]+)')  
hashtag_regex = re.compile('(#[A-Za-z0-9_]+)')

#pre-process each line i.e., tweet individually
with open(output_fname_path, 'w') as outfile:
    outfile.write("[\n")
    for line in content:
        #extract the content of the tweet
        line_content = line.split("\"")
        tweet_id = line_content[1]
        tweet_content = line_content[3]
        
        #remove HTML tags -- so far no HTML tags encountered in the dataset
        
        #remove new lines
        tweet_content = tweet_content.replace('\\n','')
    
        #remove URLs, Handles and Hashtags
        tweet_content = handle_regex.sub('', tweet_content)
        tweet_content = hashtag_regex.sub('', tweet_content)
        tweet_content = url_regex.sub('', tweet_content)
        
        #remove punctuations from the tweets
        tweet_content = punctuation_regex.sub('', tweet_content)
        #remove digits from the tweets
        tweet_content = digits_regex.sub('', tweet_content)
        #remove emojis from tweets
        tweet_content = emoji_regex.sub('', tweet_content)
        
        #write the tweet to file if only it has some content in it
        tweet_content = tweet_content.strip()
        if tweet_content and not(tweet_content.isspace()):
            tweet_dict = {"id":tweet_id, "content":tweet_content}
            tweet_json = json.dumps(tweet_dict,ensure_ascii=False)
            outfile.write(tweet_json+",\n")
#remove the last comma and close the json array
with open(output_fname_path, 'rb+') as outfile:
    outfile.seek(-2, os.SEEK_END)
    outfile.truncate()
    outfile.write("\n]".encode())